In [4]:
import pandas as pd

In [5]:
#Reading in a short story as text sample into Python
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
          raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20398
﻿I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no


### Preprocessing Steps (Chapter 2)

#### 1. Tokenizing Text

In [8]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [9]:
#separting punction and spaces from the text
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [10]:
#remvoing white spaces
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [11]:
#extending the tokenizer to handle wider range of punctuations
text = "Hello, world. Is this-- a test."
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print("Tokenized text: ",result)

Tokenized text:  ['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '.']


In [12]:
#applying this tokenizer to the full Edith Warton's story
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f"length of the complete tokenized text from Edith Wharton story: {len(preprocessed)}")

length of the complete tokenized text from Edith Wharton story: 4690


In [13]:
#printing first 30 tokens
print(f"First 30 tokens: {preprocessed[:30]}")

First 30 tokens: ['\ufeffI', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


#### 2. Converting token into token IDs

create a list of all unique tokens and sort them alphabetically to determine the vocabulary

In [16]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(f"Vocabulary Size: {vocab_size}")

Vocabulary Size: 1131


In [17]:
#creating a vocabulary
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [18]:
#implementing a tokenizer class to tokenize text into tokens, encode tokens to integers and decode integers to tokens

class SimpleTokenzierV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #A
        self.int_to_str = {i:s for s,i in vocab.items()} #B

    def encode(self, text): #C
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids): #D
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #E
        return text

In [19]:
#instantiating tokenizer object to test the SimpleTokenzierV1 class
tokenizer = SimpleTokenzierV1(vocab)

In [20]:
text = raw_text[:99]
ids = tokenizer.encode(text)
print(f"Token ids for sample text from Edith Warton's story: {ids}")

Token ids for sample text from Edith Warton's story: [1130, 44, 149, 1003, 57, 38, 818, 115, 256, 486, 6, 1002, 115, 500, 435, 392, 6, 908, 585, 1077, 709]


In [21]:
#testing decoder on the token ids above
print(f"text from decoding token ids using SimpleTokenzierV1 class: \n{tokenizer.decode(ids)}")

text from decoding token ids using SimpleTokenzierV1 class: 
﻿I HAD always thought Jack Gisburn rather a cheap genius -- though a good fellow enough -- so it was no


In [22]:
#running the tokenizer on a sample text which is not a part of the text used to create the vocab
text = "Hello, do you like tea?"
tokenizer.encode(text)

KeyError: 'Hello'

here, the key error suggests that Hello is not a part of the vocabulary and hence we need to use large and diverse text in order to extend the vocabulary when creating a LARGE language models

#### 3. Adding special context tokens

modifying SimpleTokenzierV1 to support new tokens for unknown words and document boundaries

In [53]:
'''adding tokens:
1. <unk> - to represent unknown or new words that are not part of the vocabulary
2. <|endoftext|> -  marker to separate two different text source from each other
'''
all_tokens =  sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<unk>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print(f"length of the vocabulary after extending the vocab with unknown words and end of text markers: {len(vocab.items())}")

length of the vocabulary after extending the vocab with unknown words and end of text markers: 1133


In [54]:
#printing a sample from the end of the extended vocab
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('your', 1128)
('yourself', 1129)
('\ufeffI', 1130)
('<|endoftext|>', 1131)
('<unk>', 1132)


In [55]:
#SimpleTokenzierV2 replaces unknown words with the token "<unk>"
class SimpleTokenzierV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                       else "<unk>" for item in preprocessed] #A
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #B
        return text

In [56]:
#concatenating two unrelated texts
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))
print(f"concatenated text with endoftext marker: \n{text}")

concatenated text with endoftext marker: 
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [57]:
#testing the SimpleTokenzierV2 on a text that is concatenation of two unrelated texts
tokenizer = SimpleTokenzierV2(vocab)
print(tokenizer.encode(text))

[1132, 5, 355, 1126, 628, 975, 10, 1131, 55, 988, 956, 984, 722, 988, 1132, 7]


we can see that the list of token IDs contains 1131 for the <|endoftext|> separator token as well as two 1132 tokens, which are used for unknown words.

In [59]:
#testing detokenizer for SimpleTokenzierV2
print(tokenizer.decode(tokenizer.encode(text)))

<unk>, do you like tea? <|endoftext|> In the sunlit terraces of the <unk>.


#### 4. BPE: Byte pair encoding

unlike the simple tokenizer implemented above, GPT uses byte pair encoding tokenizer which does not replace unknown words with "unk" token but breaks down the word into subword units

In [62]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [63]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In [64]:
tokenizer = tiktoken.get_encoding("gpt2") #similar to SimpleTokenzierV2

In [65]:
#encoding using BPE
text = "Hello, do you like tea? <|endoftext|> In the sunlit terra"
integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 1059, 430]


here, the tokenzier handles unknown words by breaking them down into subword units or even individual characters provinding them the largest token id

In [67]:
#decoding using BPE
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terra


In [68]:
text = "Akwirw ier"
integers = tokenizer.encode(text)
strings = tokenizer.decode(integers)
print(f"token id of unknown words broken down into subword units: {integers}")
print(f"text of unknown words from token ids: {strings}")

token id of unknown words broken down into subword units: [33901, 86, 343, 86, 220, 959]
text of unknown words from token ids: Akwirw ier


BPE is able to handle unknown words by building a vocab by iteratively merging frequent characters into subwords and frequest subwords into words

#### 5. Data Sampling with a sliding window

In [71]:
#tokenizing the entire Edith Warton story usinf the BPE tokenizer
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(f"length of tokenized Edith Warton Story {len(enc_text)}")

length of tokenized Edith Warton Story 5066


In [72]:
#taking a sample of first 50 tokens
enc_sample = enc_text[50:]

In [73]:
#creating input-target pairs for next word prediction
context_size = 4 #A

x = enc_sample[:context_size] #contains the input tokens
y = enc_sample[1:context_size+1] #contains targets' inputs shiftd by 1

print(f"x: {x}")
print(f"y:        {y}")

x: [5527, 27075, 11, 290]
y:        [27075, 11, 290, 4920]


In [74]:
#creating the next-word prediction
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "------>" , desired)

[5527] ------> 27075
[5527, 27075] ------> 11
[5527, 27075, 11] ------> 290
[5527, 27075, 11, 290] ------> 4920


In [75]:
#creating the next-word prediction - visualizing the decoded text
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "------>" , tokenizer.decode([desired]))

 rich ------>  widow
 rich widow ------> ,
 rich widow, ------>  and
 rich widow, and ------>  established


In [76]:
#implementing a data loader that iterates over the input dataset and returns inputs & targets as pytorch tensors
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) #A

        for i in range(0, len(token_ids) - max_length, stride): #B
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self): #C
        return len(self.input_ids)

    def __getitem__(self, idx): #D
        return self.input_ids[idx], self.target_ids[idx]

In [77]:
#loading the input in batches via PyTorch DataLoader
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True):
    tokenizer = tiktoken.get_encoding("gpt2") #A
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) #B
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = shuffle, drop_last = drop_last)
    return dataloader

In [78]:
#testing create_dataloader_v1 with batch_size with 1 for an LLM with context size of 4
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size = 1, max_length = 4, stride = 1, shuffle = False, drop_last = False)
data_iter = iter(dataloader) #A
first_batch = next(data_iter)
print(first_batch)

[tensor([[171, 119, 123,  40]]), tensor([[119, 123,  40, 367]])]


In [79]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[119, 123,  40, 367]]), tensor([[ 123,   40,  367, 2885]])]


- if we compare the first and the second batch - we can observe that batch token IDs in the second batch have shifted by one position as compared to the first batch
- stride setting determines the number of positons the tokens shift across batches, thereby emulating a sliding window approach
- if the stride is set equal to the input window size it prevents overlap between batches

In [81]:
#using the dataloader to sample with a batch size>1
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = 4, shuffle = False, drop_last = False)
data_iter = iter(dataloader) #A
inputs, targets = next(data_iter)
print(f"Inputs: \n{inputs}")
print(f"Targets: \n{targets}")

Inputs: 
tensor([[  171,   119,   123,    40],
        [  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284]])
Targets: 
tensor([[  119,   123,    40,   367],
        [ 2885,  1464,  1807,  3619],
        [  402,   271, 10899,  2138],
        [  257,  7026, 15632,   438],
        [ 2016,   257,   922,  5891],
        [ 1576,   438,   568,   340],
        [  373,   645,  1049,  5975],
        [  284,   502,   284,  3285]])


increasing the stride to 4 ensure no word is skipped and avoids any overlap between batches (more overlap increase overfitting)

#### 6. Creating token embeddings

Vector representation of token IDs along with token positions (absolute positional embedding) that will serve as an input data format for LLMs

In [85]:
#taking a sample of four input tokens
input_ids = torch.tensor([2, 3, 5, 1])

In [86]:
#taking a vocab size of 6 and embedding size of 3
vocab_size = 6
output_dim = 3

In [87]:
'''taking the vocab_size and output_dim to instantiate a embedding layer in PyTorch 
(setting random seed to 123 for reproducibility)'''

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- the above matrix is of 6x3 with 6 rows one for each token and 3 columns one for each embedding dimension
- these random values are further optimized in the LLM training process via backpropagation

In [89]:
#applying the instantiated embedding layer to a token ID to obtain the embedding vector
print(embedding_layer(torch.tensor([3]))) #lookup the embedding vector for this token id in the embedding layer weight matrix above

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [90]:
#applying the instantiated embedding layer to all token ID to obtain the embedding vector
print(embedding_layer(input_ids)) 

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


#### 7. Encoding word positions

In [92]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [93]:
#instantiating the data loader
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = max_length, stride = max_length, 
                                  shuffle = False, drop_last = False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Token IDs: \n{inputs}")
print(f"Inputs shape: \n{inputs.shape}")

Token IDs: 
tensor([[  171,   119,   123,    40],
        [  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284]])
Inputs shape: 
torch.Size([8, 4])


the token ID tensor of shape 8X4 indicates that the data batch consists 8 text samples with 4 tokens each

In [95]:
#using the embedding layer to embed these token ids in 256 dimensional vectors
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [96]:
'''following the GPT model's absolute positional embedding approach - creating another embedding layer 
    with same dimension as token_embedding_layer'''

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [97]:
#adding the pos_embeddings to token_embeddings in each of the 8 batches
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


these input_embeddings are now ready to be processed by the main LLM Module

### Coding attention mechanism (Chapter 3)

In [100]:
#taking a sample input sentence
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1) 
 [0.55, 0.87, 0.66], # journey (x^2)
 [0.57, 0.85, 0.64], # starts (x^3)
 [0.22, 0.58, 0.33], # with (x^4)
 [0.77, 0.25, 0.10], # one (x^5)
 [0.05, 0.80, 0.55]] # step (x^6)
)

#### 8. Computing intermediate attention scores

In [102]:
#computing intermediate attention scores between query and input token by taking dot product of query with input token
query = inputs[1] #A
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(f"the computed intermediate attention scores are: \n{attn_scores_2}")

the computed intermediate attention scores are: 
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


#### 9. Obtaining attention weights

In [104]:
#obtaining attention weights (that sum up to 1) by normalizing the attention scores
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(f"attention weights: {attn_weights_2_tmp}")
print(f"sum of attention weights: {attn_weights_2_tmp.sum()}")

attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
sum of attention weights: 1.0000001192092896


In [105]:
'''softmax function is commonly used to normalize the attention scores 
(can handle extreme values, ensures attn. weights are alays positive etc.)'''
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(f"attention weights: {attn_weights_2_naive}")
print(f"sum of attention weights: {attn_weights_2_naive.sum()}")

attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum of attention weights: 1.0


In [106]:
#using PyTorch softmax funtion to prevent overflow and underflow while dealing with large or small input values
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
print(f"attention weights: {attn_weights_2}")
print(f"sum of attention weights: {attn_weights_2.sum()}")

attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum of attention weights: 1.0


#### 10. Calculating context vector

In [108]:
'''calculatin context vector by multiplying embedded input tokens with corresponding attention weights 
and then summing the resulting vectors'''
query = inputs[1] #second input token is the query
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [109]:
#extending the above implementation to calculate the attention weights and context vector for all inputs
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [110]:
#getting the attn scores without for loop
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [111]:
#normalizing each row in the above output to sum up the values to 1
attn_weights  = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [112]:
#verifying the normalization
row_2_sum = sum([0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452])
print(f"Row 2 sum: {row_2_sum}")
print(f"All row sum: {attn_weights.sum(dim=1)}")

Row 2 sum: 0.9999
All row sum: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [113]:
#getting context vectors
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [114]:
#cross validating th context vecs with the previously generated context vec for second row
print(f"Previous 2nd context vector: {context_vec_2}")

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


#### 11. Implementing self-attention with trainable weights
Enable LLM to learn from data and improve its performance on specific tasks

In [116]:
#computing attention weights step by step
x_2 = inputs[1] #A
d_in = inputs.shape[1] #B
d_out = 2 #C

In [117]:
#intializing 3 weight metrics
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad =  False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad =  False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad =  False)

In [118]:
#computing query, key, and value vectors
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [119]:
#obtaining all keys and values
keys = inputs @ W_key
value = inputs @ W_value
print(f"keys.shape: {keys.shape}")
print(f"value.shape: {value.shape}")

keys.shape: torch.Size([6, 2])
value.shape: torch.Size([6, 2])


In [120]:
#computing attention scores
keys_2 = keys[1] #A
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [121]:
#applying across all attention scores
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [122]:
'''normalizing the attention scores using softmax to get attention weights.
scaling attention scores by dividing them by square root (or exponentiating by 0.5) of the embedding dimension of the keys
'''
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim = -1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [123]:
#computing context vectors
context_vec_2 = attn_weights_2 @ value
print(context_vec_2)

tensor([0.3061, 0.8210])


In [124]:
# implementing a compact self-attention python class
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        '''initializes trainable weights for:
        queries, keys, values -- each transforming the input 
        with dimension d_in to dimension d_out
        '''
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        '''computing attention scores'''
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T #omega
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim = -1)
        context_vec = attn_weights @ values
        return context_vec

In [125]:
#using the above class
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


the second row here matches with the context_vec_2

In [127]:
#self-attention class using PyTorch's Linear layers (has optimized weight intialization scheme)
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim = -1)
        context_vec = attn_weights @ values
        return context_vec

In [128]:
#using the above class
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


both sa_v1 and sa_v2 have different outputs because they use different initial weights

#### 12. Hiding future words with casual attention (masked attention)

In [131]:
#computing attention weights using softmax function
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim = -1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [132]:
#using PyTorch tril function to create a mask where values above the diagnol are zero
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [133]:
#multiplying the mask with attention weights to zero out the values above the diagnol
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [134]:
#renormalizing the attention weights to sum up the weight to 1 in each row
row_sums = masked_simple.sum(dim = 1, keepdim = True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [138]:
#masking attention scores with negative infinity values
mask = torch.triu(torch.ones(context_length, context_length), diagonal = 1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [140]:
#applying softmax function on the masked results to normalize each row
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim = 1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


Softmax function converts intputs to probability distribution and treats negative infinity values as zero probability

#### 13. Masking additonal attention weights with dropout

DL technique where randomly selected layers are ignored during training (dropping them out)

In [145]:
#implementing drop out rate of 50% - masking 50% of attention weights
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) #A
example = torch.ones(6, 6) #B
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


- random elements are set to zero and the remaining values as scaled by a factor of 1/0.5 to compensate for this reduction in elements.
- this scaling helps maintain the average influence of the attention mechanism during both training and inference

In [148]:
#applyinf drop out to attention weight matrix
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


In [ ]:
#implementing compact casual attention class
